In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib notebook
sns.set()

In [ ]:
import galsim

from mdetsims import Sim
from metadetect.detect import MEDSifier
from mdetsims.defaults import TEST_METACAL_MOF_CONFIG
from mdetsims.metacal import get_fofs, MOFFitter, MetacalFitter
from mdetsims.metacal.util import get_masked_frac

In [ ]:
des_sim = Sim(
    rng=np.random.RandomState(seed=100),
    g1=0.02,
    gal_type='wldeblend',
    scale=0.263,
    psf_type='gauss',
    n_coadd=30,
    n_coadd_psf=1,
    gal_kws={'survey_name': 'DES'})

In [ ]:
mbobs = des_sim.get_mbobs()

In [ ]:
mfier = MEDSifier(
    mbobs=mbobs,
    sx_config=TEST_METACAL_MOF_CONFIG['sx'],
    meds_config=TEST_METACAL_MOF_CONFIG['meds'])

In [ ]:
mbmeds = mfier.get_multiband_meds()

## Try the FoFs

In [ ]:
res = get_fofs(mfier.cat, TEST_METACAL_MOF_CONFIG['fofs'])

In [ ]:
n_fofs = np.max(res[1]['fofid'])

In [ ]:
for fofid in range(n_fofs):
    msk = res[1]['fofid'] == fofid
    if np.sum(msk) > 2:
        inds = res[1]['number'][msk] - 1
        list_of_mbobs = []
        for ind in inds:
            o = mbmeds.get_mbobs(ind)
            o[0].meta['Tsky'] = 1
            o.meta['masked_frac'] = get_masked_frac(o)
            o[0].meta['magzp_ref'] = 26.5
            o.meta['id'] = ind + 1
            o.meta['fofid'] = fofid
            list_of_mbobs.append(o)

In [ ]:
list_of_mbobs

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(2, 2, 1)
sns.heatmap(np.arcsinh(list_of_mbobs[0][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 2)
sns.heatmap(np.arcsinh(list_of_mbobs[1][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 3)
sns.heatmap(np.arcsinh(list_of_mbobs[2][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 4)
sns.heatmap(np.arcsinh(list_of_mbobs[3][0][0].image), square=True, ax=ax)

## Try MOF

In [ ]:
fitter = MOFFitter(TEST_METACAL_MOF_CONFIG, 1, np.random.RandomState(seed=10))

In [ ]:
od, ed = fitter.go(list_of_mbobs)

In [ ]:
mof_fitter = fitter.get_mof_fitter()

In [ ]:
rs = mof_fitter.make_corrected_obs()

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(2, 2, 1)
sns.heatmap(np.arcsinh(rs[0][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 2)
sns.heatmap(np.arcsinh(rs[1][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 3)
sns.heatmap(np.arcsinh(rs[2][0][0].image), square=True, ax=ax)

ax = fig.add_subplot(2, 2, 4)
sns.heatmap(np.arcsinh(rs[3][0][0].image), square=True, ax=ax)

## Try Metacal + MOF

In [ ]:
rng = np.random.RandomState(seed=10)
nband = 1
mcal = MetacalFitter(
    TEST_METACAL_MOF_CONFIG, nband, rng, 
    mof_fitter=MOFFitter(TEST_METACAL_MOF_CONFIG, nband, rng))

In [ ]:
mcal.go(list_of_mbobs)

In [ ]:
res = mcal.result

In [ ]:
res['flags']